# Training

This notebook demonstrates the process of training an ensemble learning model using a provided CSV file. It showcases data preprocessing, model training, evaluation, and saving the trained model. The ensemble method (hard voting, soft voting, or stacking) can be selected based on the user's choice.


## Importing Necessary Libraries

First, we import all the necessary libraries and modules needed for this script. This includes libraries for handling warnings, data manipulation, machine learning, and the custom Ensemble module containing ensemble learning methods.

In [ ]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, 
    recall_score, 
    precision_score, 
    f1_score
)
from src import Ensemble

## Utility Functions

### Function to Read CSV File

The `read_csv_file` function reads the CSV file and returns a pandas DataFrame. If the file is not found, the script will exit with an error message.

In [ ]:
def read_csv_file(filename: str) -> pd.DataFrame:
    try:
        data = pd.read_csv(filename, lineterminator='\n', dtype={'label': float})
        print("CSV file read successfully!")
        return data
    except FileNotFoundError:
        print("ERROR: File not found")
        exit(1)

# Demonstrate reading a CSV file (use a sample or mock filename)
read_csv_file('data.csv')

### Function to Seed Random Number Generators

To ensure reproducibility, the `seed_random_number_generators` function seeds the random number generators for PyTorch and NumPy.

In [ ]:
def seed_random_number_generators(seed=0):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    print("Random number generators seeded.")

# Seed the random number generators
seed_random_number_generators()

### Function for Train-Test Split

The `get_train_test_split` function splits the dataset into training and testing sets with an 80-20 split ratio and returns them.

In [ ]:
def get_train_test_split(data_frame: pd.DataFrame):
    text = data_frame['text']
    labels = data_frame['label'].to_numpy()

    X_train, X_test, y_train, y_test = train_test_split(text, labels, test_size=0.2, random_state=42)
    print("Data split into training and testing sets.")
    return X_train, X_test, y_train, y_test

# Demonstrate train-test split (use a sample DataFrame)
df = pd.DataFrame({'text': ['sample1', 'sample2', 'sample3', 'sample4'], 'label': [0, 1, 0, 1]})
get_train_test_split(df)  # Uncomment this line to test

## Ensemble Training Function

The `train_ensemble` function initializes and trains the ensemble model using the provided training data. It takes the training features and labels as input, along with the ensemble model instance, and returns the trained model.

In [ ]:
def train_ensemble(X_train: list, y_train: list, ensemble):
    seed_random_number_generators()  # Ensure reproducibility
    ensemble.fit(X_train, y_train)
    print("Ensemble model trained.")
    return ensemble

## Prediction and Evaluation Function

The `get_prediction_results` function uses the trained ensemble model to make predictions on the test set and then evaluates these predictions by calculating the accuracy, recall, precision, and F1-score. It returns these metrics for further analysis.

In [ ]:
def get_prediction_results(X_test: list, y_test: list, ensemble):
    with torch.inference_mode():
        y_pred = ensemble.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        print(f"Accuracy: {accuracy}\nRecall: {recall}\nPrecision: {precision}\nF1-score: {f1}")
        return accuracy, recall, precision, f1

## Save Model Function

The `save_trained_model` function saves the trained ensemble model to disk using the joblib library. This allows for the model to be reloaded and used for predictions without the need for retraining.

In [ ]:
def save_trained_model(ensemble, filename="Ensemble"):
    import joblib
    joblib.dump(ensemble, f'{filename}.pkl', compress=True)
    print(f"Ensemble model saved to {filename}.pkl")

## Main Execution Workflow

This cell combines all the previous steps to execute the workflow. It includes reading the dataset, splitting it into training and testing sets, selecting the ensemble method, training the model, evaluating its performance, and saving the trained model. Replace 'your_dataset.csv' with the path to your dataset and choose an appropriate ensemble method.

In [ ]:
# Replace 'your_dataset.csv' with your actual dataset path
FILENAME = 'your_dataset.csv'  # Adjust to your dataset's filename
# Choose from 'hard', 'soft', 'stacking' for your ensemble method
ENSEMBLE_METHOD = 'hard'  # Adjust to your chosen ensemble method

# Read data and prepare train-test split
data_frame = read_csv_file(FILENAME)
X_train, X_test, y_train, y_test = get_train_test_split(data_frame)

# Initialize and train the ensemble
ensemble_methods = {
    'hard': Ensemble.HardVotingEnsemble(),
    'soft': Ensemble.SoftVotingEnsemble(),
    'stacking': Ensemble.StackingEnsemble(),
}
ensemble = train_ensemble(X_train, y_train, ensemble_methods[ENSEMBLE_METHOD])

# Evaluate the trained ensemble and display results
accuracy, recall, precision, f1 = get_prediction_results(X_test, y_test, ensemble)

# Save the trained model
save_trained_model(ensemble, f'ensemble-{ENSEMBLE_METHOD}')

## Results

To better visualize the evaluation results, this cell creates a pandas DataFrame to display the accuracy, recall, precision, and F1-score in a tabular format.

In [ ]:
import pandas as pd
results_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Recall', 'Precision', 'F1-Score'],
    'Value': [accuracy, recall, precision, f1]
})
results_df